In [ ]:
function KeepClicking(){
console.log("Clicking");
document.querySelector("colab-connect-button").click()
}
setInterval(KeepClicking,60000)
Open your Chrome DevTools by pressing F12 or ctrl+shift+i on Linux and enter the following JavaScript snippet in your console:

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# install transformers
!pip install pytorch_transformers
# Mount google drive
!pip install -r drive/MyDrive/transformers_cl/requirements.txt

In [ ]:
%cd 'drive/MyDrive/transformers_cl'

/content/drive/MyDrive/transformers_cl


In [ ]:
from __future__ import absolute_import, division, print_function
from random import sample
import argparse
import glob
import logging
import os
import random

import numpy as np
import torch
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset, Subset)
from torch.utils.data.distributed import DistributedSampler
from tensorboardX import SummaryWriter
from tqdm import tqdm, trange

from pytorch_transformers import (WEIGHTS_NAME, BertConfig,
                                  BertForSequenceClassification, BertTokenizer,
                                  XLMConfig, XLMForSequenceClassification,
                                  XLMTokenizer, XLNetConfig,
                                  XLNetForSequenceClassification,
                                  XLNetTokenizer)

from pytorch_transformers import AdamW, WarmupLinearSchedule

from utils_glue import (compute_metrics, convert_examples_to_features,
                        output_modes, processors, read_curriculum_file, cycle,
                        compute_aps, read_scores_file)

from pacing_functions import (PACING_FUNCTIONS)
from IPython import embed
from scipy.special import softmax

In [ ]:
cached_features_file = "../Mantis/cached_dev_bert-base-uncased_128_mantis_10"
features = torch.load(cached_features_file)

In [ ]:
import numpy as np
np.shape(features)

(180960,)

In [ ]:
sample(features, 10)

# Mantis root-2

In [ ]:
# seed 1 60000
!python drive/MyDrive/transformers_cl/run_glue.py \
    --model_type bert \
    --model_name_or_path bert-base-uncased \
    --task_name  mantis_10 \
    --do_train \
    --evaluate_during_training \
    --do_lower_case \
    --data_dir drive/MyDrive/Mantis \
    --max_seq_length 128 \
    --per_gpu_eval_batch_size=64   \
    --per_gpu_train_batch_size=64   \
    --learning_rate 2e-5 \
    --num_train_epochs 1 \
    --seed 1\
    --output_dir drive/MyDrive/transformers_cl/mantis_10_root2_seed_1_8_6w \
    --logging_steps 300 \
    --curriculum_file  drive/MyDrive/transformers_cl/mantis_10_bert_bm25_8/preds_dif_c_values_3 \
    --pacing_function root_2\
    --use_additive_cl \
    --eval_all_checkpoints \
    --invert_cl_values\
    --eval_subsize 60000\

08/09/2021 21:09:41 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
08/09/2021 21:09:41 - INFO - pytorch_transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json not found in cache or force_download set to True, downloading to /tmp/tmpf527geb_
100% 433/433 [00:00<00:00, 440274.82B/s]
08/09/2021 21:09:41 - INFO - pytorch_transformers.file_utils -   copying /tmp/tmpf527geb_ to cache at /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
08/09/2021 21:09:41 - INFO - pytorch_transformers.file_utils -   creating metadata file for /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
08/09/2021 21:09:41 - INFO - pytorch_transformers.file_u

In [ ]:
# seed 2 60000
!python drive/MyDrive/transformers_cl/run_glue.py \
    --model_type bert \
    --model_name_or_path bert-base-uncased \
    --task_name  mantis_10 \
    --do_train \
    --evaluate_during_training \
    --do_lower_case \
    --data_dir drive/MyDrive/Mantis \
    --max_seq_length 128 \
    --per_gpu_eval_batch_size=64   \
    --per_gpu_train_batch_size=64   \
    --learning_rate 2e-5 \
    --num_train_epochs 1 \
    --seed 2\
    --output_dir drive/MyDrive/transformers_cl/mantis_10_root2_seed_2_8_6w \
    --logging_steps 300 \
    --curriculum_file  drive/MyDrive/transformers_cl/mantis_10_bert_bm25_8/preds_dif_c_values_3 \
    --pacing_function root_2\
    --use_additive_cl \
    --eval_all_checkpoints \
    --invert_cl_values\
    --eval_subsize 60000\

08/10/2021 13:39:33 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
08/10/2021 13:39:34 - INFO - pytorch_transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json not found in cache or force_download set to True, downloading to /tmp/tmp6p9ags_1
100% 433/433 [00:00<00:00, 420206.76B/s]
08/10/2021 13:39:34 - INFO - pytorch_transformers.file_utils -   copying /tmp/tmp6p9ags_1 to cache at /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
08/10/2021 13:39:34 - INFO - pytorch_transformers.file_utils -   creating metadata file for /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
08/10/2021 13:39:34 - INFO - pytorch_transformers.file_u

In [ ]:
!python drive/MyDrive/transformers_cl/run_glue.py \
    --model_type bert \
    --model_name_or_path bert-base-uncased \
    --task_name  mantis_10 \
    --do_train \
    --evaluate_during_training \
    --do_lower_case \
    --data_dir drive/MyDrive/Mantis \
    --max_seq_length 128 \
    --per_gpu_eval_batch_size=64   \
    --per_gpu_train_batch_size=64   \
    --learning_rate 2e-5 \
    --num_train_epochs 1 \
    --seed 3\
    --output_dir drive/MyDrive/transformers_cl/mantis_10_root2_seed_3_8_6w \
    --logging_steps 300 \
    --curriculum_file  drive/MyDrive/transformers_cl/mantis_10_bert_bm25_8/preds_dif_c_values_3 \
    --pacing_function root_2\
    --use_additive_cl \
    --eval_all_checkpoints \
    --invert_cl_values\
    --eval_subsize 60000\

08/10/2021 20:07:45 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
08/10/2021 20:07:45 - INFO - pytorch_transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json not found in cache or force_download set to True, downloading to /tmp/tmpucng8vzu
100% 433/433 [00:00<00:00, 513757.75B/s]
08/10/2021 20:07:46 - INFO - pytorch_transformers.file_utils -   copying /tmp/tmpucng8vzu to cache at /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
08/10/2021 20:07:46 - INFO - pytorch_transformers.file_utils -   creating metadata file for /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
08/10/2021 20:07:46 - INFO - pytorch_transformers.file_u

# Mantis root-5

In [ ]:
# seed 1 6w
!python drive/MyDrive/transformers_cl/run_glue.py \
    --model_type bert \
    --model_name_or_path bert-base-uncased \
    --task_name  mantis_10 \
    --do_train \
    --evaluate_during_training \
    --do_lower_case \
    --data_dir drive/MyDrive/Mantis \
    --max_seq_length 128 \
    --per_gpu_eval_batch_size=64   \
    --per_gpu_train_batch_size=64   \
    --learning_rate 2e-5 \
    --num_train_epochs 1 \
    --seed 1\
    --output_dir drive/MyDrive/transformers_cl/mantis_10_root5_seed_1_8_6w \
    --logging_steps 300 \
    --curriculum_file  drive/MyDrive/transformers_cl/mantis_10_bert_bm25_8/preds_dif_c_values_3 \
    --pacing_function root_5\
    --use_additive_cl \
    --eval_all_checkpoints \
    --invert_cl_values\
    --eval_subsize 60000\

08/09/2021 13:36:19 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
08/09/2021 13:36:19 - INFO - pytorch_transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json not found in cache or force_download set to True, downloading to /tmp/tmpcb7_b74e
100% 433/433 [00:00<00:00, 406839.97B/s]
08/09/2021 13:36:20 - INFO - pytorch_transformers.file_utils -   copying /tmp/tmpcb7_b74e to cache at /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
08/09/2021 13:36:20 - INFO - pytorch_transformers.file_utils -   creating metadata file for /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
08/09/2021 13:36:20 - INFO - pytorch_transformers.file_u

In [ ]:
# seed 1 6 h for 2 epochs
!python drive/MyDrive/transformers_cl/run_glue.py \
    --model_type bert \
    --model_name_or_path bert-base-uncased \
    --task_name  mantis_10 \
    --do_train \
    --evaluate_during_training \
    --do_lower_case \
    --data_dir drive/MyDrive/Mantis \
    --max_seq_length 128 \
    --per_gpu_eval_batch_size=64   \
    --per_gpu_train_batch_size=64   \
    --learning_rate 2e-5 \
    --num_train_epochs 1 \
    --seed 2\
    --output_dir drive/MyDrive/transformers_cl/mantis_10_root5_seed_2_8_6w \
    --logging_steps 300 \
    --curriculum_file  drive/MyDrive/transformers_cl/mantis_10_bert_bm25_8/preds_dif_c_values_3 \
    --pacing_function root_5\
    --use_additive_cl \
    --eval_all_checkpoints \
    --invert_cl_values\
    --eval_subsize 60000\

08/10/2021 14:29:17 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
08/10/2021 14:29:18 - INFO - pytorch_transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json not found in cache or force_download set to True, downloading to /tmp/tmpikzo_bn5
100% 433/433 [00:00<00:00, 444912.70B/s]
08/10/2021 14:29:18 - INFO - pytorch_transformers.file_utils -   copying /tmp/tmpikzo_bn5 to cache at /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
08/10/2021 14:29:18 - INFO - pytorch_transformers.file_utils -   creating metadata file for /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
08/10/2021 14:29:18 - INFO - pytorch_transformers.file_u

In [ ]:
# seed 2 60000
!python drive/MyDrive/transformers_cl/run_glue.py \
    --model_type bert \
    --model_name_or_path bert-base-uncased \
    --task_name  mantis_10 \
    --do_train \
    --evaluate_during_training \
    --do_lower_case \
    --data_dir drive/MyDrive/Mantis \
    --max_seq_length 128 \
    --per_gpu_eval_batch_size=64   \
    --per_gpu_train_batch_size=64   \
    --learning_rate 2e-5 \
    --num_train_epochs 1 \
    --seed 3\
    --output_dir drive/MyDrive/transformers_cl/mantis_10_root5_seed_3_8_6w \
    --logging_steps 300 \
    --curriculum_file  drive/MyDrive/transformers_cl/mantis_10_bert_bm25_8/preds_dif_c_values_3 \
    --pacing_function root_5\
    --use_additive_cl \
    --eval_all_checkpoints \
    --invert_cl_values\
    --eval_subsize 60000\

08/11/2021 09:09:12 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
08/11/2021 09:09:12 - INFO - pytorch_transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json not found in cache or force_download set to True, downloading to /tmp/tmp4iw4_nlc
100% 433/433 [00:00<00:00, 468437.87B/s]
08/11/2021 09:09:13 - INFO - pytorch_transformers.file_utils -   copying /tmp/tmp4iw4_nlc to cache at /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
08/11/2021 09:09:13 - INFO - pytorch_transformers.file_utils -   creating metadata file for /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
08/11/2021 09:09:13 - INFO - pytorch_transformers.file_u

# Mantis root-10

In [ ]:
# seed_1 6w
!python drive/MyDrive/transformers_cl/run_glue.py \
    --model_type bert \
    --model_name_or_path bert-base-uncased \
    --task_name  mantis_10 \
    --do_train \
    --evaluate_during_training \
    --do_lower_case \
    --data_dir drive/MyDrive/Mantis \
    --max_seq_length 128 \
    --per_gpu_eval_batch_size=64   \
    --per_gpu_train_batch_size=64   \
    --learning_rate 2e-5 \
    --num_train_epochs 1 \
    --seed 1\
    --output_dir drive/MyDrive/transformers_cl/mantis_10_root10_seed_1_8_6w \
    --logging_steps 300 \
    --curriculum_file  drive/MyDrive/transformers_cl/mantis_10_bert_bm25_8/preds_dif_c_values_3 \
    --pacing_function root_10\
    --use_additive_cl \
    --eval_all_checkpoints \
    --invert_cl_values\
    --eval_subsize 60000\

08/09/2021 15:43:32 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
08/09/2021 15:43:32 - INFO - pytorch_transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json not found in cache or force_download set to True, downloading to /tmp/tmpwy3t4iwe
100% 433/433 [00:00<00:00, 435002.07B/s]
08/09/2021 15:43:32 - INFO - pytorch_transformers.file_utils -   copying /tmp/tmpwy3t4iwe to cache at /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
08/09/2021 15:43:32 - INFO - pytorch_transformers.file_utils -   creating metadata file for /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
08/09/2021 15:43:32 - INFO - pytorch_transformers.file_u

In [ ]:
# seed_1
!python drive/MyDrive/transformers_cl/run_glue.py \
    --model_type bert \
    --model_name_or_path bert-base-uncased \
    --task_name  mantis_10 \
    --do_train \
    --evaluate_during_training \
    --do_lower_case \
    --data_dir drive/MyDrive/Mantis \
    --max_seq_length 128 \
    --per_gpu_eval_batch_size=64   \
    --per_gpu_train_batch_size=64   \
    --learning_rate 2e-5 \
    --num_train_epochs 1 \
    --seed 2\
    --output_dir drive/MyDrive/transformers_cl/mantis_10_root_10_seed_2_8_6w \
    --logging_steps 300 \
    --curriculum_file  drive/MyDrive/transformers_cl/mantis_10_bert_bm25_8/preds_dif_c_values_3 \
    --pacing_function root_10\
    --use_additive_cl \
    --eval_all_checkpoints \
    --invert_cl_values\
    --eval_subsize 60000\

08/10/2021 20:00:33 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
08/10/2021 20:00:34 - INFO - pytorch_transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json not found in cache or force_download set to True, downloading to /tmp/tmpkpymmvug
100% 433/433 [00:00<00:00, 398449.68B/s]
08/10/2021 20:00:34 - INFO - pytorch_transformers.file_utils -   copying /tmp/tmpkpymmvug to cache at /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
08/10/2021 20:00:34 - INFO - pytorch_transformers.file_utils -   creating metadata file for /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
08/10/2021 20:00:34 - INFO - pytorch_transformers.file_u

In [ ]:
# seed_3
!python drive/MyDrive/transformers_cl/run_glue.py \
    --model_type bert \
    --model_name_or_path bert-base-uncased \
    --task_name  mantis_10 \
    --do_train \
    --evaluate_during_training \
    --do_lower_case \
    --data_dir drive/MyDrive/Mantis \
    --max_seq_length 128 \
    --per_gpu_eval_batch_size=64   \
    --per_gpu_train_batch_size=64   \
    --learning_rate 2e-5 \
    --num_train_epochs 1 \
    --seed 3\
    --output_dir drive/MyDrive/transformers_cl/mantis_10_root_10_seed_3_8_6w \
    --logging_steps 300 \
    --curriculum_file  drive/MyDrive/transformers_cl/mantis_10_bert_bm25_8/preds_dif_c_values_3 \
    --pacing_function root_10\
    --use_additive_cl \
    --eval_all_checkpoints \
    --invert_cl_values\
    --eval_subsize 60000\

08/11/2021 11:38:55 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
08/11/2021 11:38:55 - INFO - pytorch_transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json not found in cache or force_download set to True, downloading to /tmp/tmp6ykc18aa
100% 433/433 [00:00<00:00, 409777.44B/s]
08/11/2021 11:38:55 - INFO - pytorch_transformers.file_utils -   copying /tmp/tmp6ykc18aa to cache at /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
08/11/2021 11:38:55 - INFO - pytorch_transformers.file_utils -   creating metadata file for /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
08/11/2021 11:38:55 - INFO - pytorch_transformers.file_u